# Tarea 2 AID

- Bruno Benkel - 201204111-2
- Felipe Chacon - 201303017-3
- Diego Wilhelm - 201303059-9

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import cluster
from sklearn import metrics
from statsmodels.sandbox.tools.tools_pca import pcasvd
from sklearn.neighbors import kneighbors_graph # Técnica de k-Vecino mas cercano
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.cross_validation import train_test_split 
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from scipy import stats, integrate
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Hepatitis

 1. Class: DIE, LIVE
 2. AGE: 10, 20, 30, 40, 50, 60, 70, 80
 3. SEX: male, female
 4. STEROID: no, yes
 5. ANTIVIRALS: no, yes
 6. FATIGUE: no, yes
 7. MALAISE: no, yes
 8. ANOREXIA: no, yes
 9. LIVER BIG: no, yes
 10. LIVER FIRM: no, yes
 11. SPLEEN PALPABLE: no, yes
 12. SPIDERS: no, yes
 13. ASCITES: no, yes
 14. VARICES: no, yes
 15. BILIRUBIN: 0.39, 0.80, 1.20, 2.00, 3.00, 4.00
 16. ALK PHOSPHATE: 33, 80, 120, 160, 200, 250
 17. SGOT: 13, 100, 200, 300, 400, 500, 
 18. ALBUMIN: 2.1, 3.0, 3.8, 4.5, 5.0, 6.0
 19. PROTIME: 10, 20, 30, 40, 50, 60, 70, 80, 90
 20. HISTOLOGY: no, yes

In [3]:
df1 = pd.read_csv('data/hepatitis.data')

In [4]:
#Quitar las filas que tengan missing values '?'
index = []
for i in range(df1.shape[0]):
    if('?' in df1.iloc[i].values):
        index.append(i)
df1 = df1.drop(index)        

In [5]:
#Sacar la columna Class, que es la que se intenta predecir
target1 = df1['Class']
df1 = df1.drop('Class',axis=1)

In [6]:
mat = df1.as_matrix()

## Clustering

Fueron utilizados K-Means, Average Linkage y Mean Shift.

### K-Means

In [7]:
kmeans = cluster.KMeans(n_clusters=2)
kmeans.fit(mat)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [8]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, kmeans.labels_))

Silhouette Coefficient: 0.482762


### Average Linkage

In [9]:
algoritmo = cluster.AgglomerativeClustering(linkage="average", 
                                            affinity="cityblock", n_clusters=2)
# Linkage: complete, average, ward
# Affinity: “euclidean”, “l1”, “l2”, “manhattan”, "cityblock", “cosine”, o ‘precomputed’
algoritmo.fit(mat)

AgglomerativeClustering(affinity='cityblock', compute_full_tree='auto',
            connectivity=None, linkage='average', memory=None,
            n_clusters=2, pooling_func=<function mean at 0x7f5454415400>)

In [10]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, algoritmo.labels_))

Silhouette Coefficient: 0.671057


### Mean Shift

In [11]:
ancho_banda = cluster.estimate_bandwidth(mat, quantile=0.15)
mean_shift_alg = cluster.MeanShift(bandwidth=ancho_banda, bin_seeding=True)
mean_shift_alg.fit(mat)

MeanShift(bandwidth=59.752845675029263, bin_seeding=True, cluster_all=True,
     min_bin_freq=1, n_jobs=1, seeds=None)

In [12]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, mean_shift_alg.labels_))

Silhouette Coefficient: 0.368808


## Clasificadores

In [13]:
H_entrenamiento, H_test, target1_entrenamiento, target1_test = train_test_split(df1, target1, test_size = 0.5, random_state=0)

sc1 = StandardScaler()
sc1.fit(H_entrenamiento) 
Z1_entrenamiento = sc1.transform(H_entrenamiento)
Z1_test = sc1.transform(H_test)

clasificador1 = LinearDiscriminantAnalysis()
clasificador1.fit(Z1_entrenamiento, target1_entrenamiento)
target1_pred1 = clasificador1.predict(Z1_test)
accuracy1 = accuracy_score(target1_test, target1_pred1)
precision1 = precision_score(target1_test, target1_pred1, average="macro")
F1 = f1_score(target1_test, target1_pred1, average='macro')

clasificador2 = QuadraticDiscriminantAnalysis()
clasificador2.fit(Z1_entrenamiento, target1_entrenamiento)
target1_pred2 = clasificador2.predict(Z1_test)
accuracy2 = accuracy_score(target1_test, target1_pred2)
precision2 = precision_score(target1_test, target1_pred2, average="macro")
F2 = f1_score(target1_test, target1_pred2, average='macro')

clasificador3=SVC(kernel="linear", C=0.025)
clasificador3.fit(Z1_entrenamiento, target1_entrenamiento)
target1_pred3 = clasificador3.predict(Z1_test)
accuracy3 = accuracy_score(target1_test, target1_pred3)
precision3 = precision_score(target1_test, target1_pred3, average="macro")
F3 = f1_score(target1_test, target1_pred3, average='macro')

print("[Clasificador 1] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy1, precision1, F1))
print("[Clasificador 2] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy2, precision2, F2))
print("[Clasificador 3] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy3, precision3, F3))

[Clasificador 1] Accuracy: 0.750  -  Precision: 0.60  -  F Score: 0.609
[Clasificador 2] Accuracy: 0.850  -  Precision: 0.42  -  F Score: 0.459
[Clasificador 3] Accuracy: 0.875  -  Precision: 0.76  -  F Score: 0.736


### KFold

In [14]:
scores1 = cross_val_score(clasificador1, df1, target1, cv=10, scoring='accuracy')
scores2 = cross_val_score(clasificador2, df1, target1, cv=10, scoring='accuracy')
scores3 = cross_val_score(clasificador3, df1, target1, cv=10, scoring='accuracy')

print("[Clasificador1] Mean: %0.3f - Standar Deviation: %0.3f"%(scores1.mean(), scores1.std()))
print("[Clasificador2] Mean: %0.3f - Standar Deviation: %0.3f"%(scores2.mean(), scores2.std()))
print("[Clasificador3] Mean: %0.3f - Standar Deviation: %0.3f"%(scores3.mean(), scores3.std()))

[Clasificador1] Mean: 0.761 - Standar Deviation: 0.181
[Clasificador2] Mean: 0.840 - Standar Deviation: 0.042
[Clasificador3] Mean: 0.814 - Standar Deviation: 0.057


## PCA

In [15]:
df1 = df1.astype(float)
df1 = (df1 - df1.mean()) / df1.std()
pca1 = pcasvd(df1, keepdim=15, demean=False)

In [16]:
H_entrenamiento, H_test, target1_entrenamiento, target1_test = train_test_split(pca1[0], target1, test_size = 0.5, random_state=0)

sc1 = StandardScaler()
sc1.fit(H_entrenamiento) 
Z1_entrenamiento = sc1.transform(H_entrenamiento)
Z1_test = sc1.transform(H_test)

clasificador1 = LinearDiscriminantAnalysis()
clasificador1.fit(Z1_entrenamiento, target1_entrenamiento)
target1_pred1 = clasificador1.predict(Z1_test)
accuracy1 = accuracy_score(target1_test, target1_pred1)
precision1 = precision_score(target1_test, target1_pred1, average="macro")
F1 = f1_score(target1_test, target1_pred1, average='macro')

clasificador2 = QuadraticDiscriminantAnalysis()
clasificador2.fit(Z1_entrenamiento, target1_entrenamiento)
target1_pred2 = clasificador2.predict(Z1_test)
accuracy2 = accuracy_score(target1_test, target1_pred2)
precision2 = precision_score(target1_test, target1_pred2, average="macro")
F2 = f1_score(target1_test, target1_pred2, average='macro')

clasificador3=SVC(kernel="linear", C=0.025)
clasificador3.fit(Z1_entrenamiento, target1_entrenamiento)
target1_pred3 = clasificador3.predict(Z1_test)
accuracy3 = accuracy_score(target1_test, target1_pred3)
precision3 = precision_score(target1_test, target1_pred3, average="macro")
F3 = f1_score(target1_test, target1_pred3, average='macro')

print("[Clasificador 1] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy1, precision1, F1))
print("[Clasificador 2] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy2, precision2, F2))
print("[Clasificador 3] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy3, precision3, F3))

[Clasificador 1] Accuracy: 0.675  -  Precision: 0.56  -  F Score: 0.551
[Clasificador 2] Accuracy: 0.850  -  Precision: 0.42  -  F Score: 0.459
[Clasificador 3] Accuracy: 0.800  -  Precision: 0.56  -  F Score: 0.543


In [17]:
scores1 = cross_val_score(clasificador1, df1, target1, cv=10, scoring='accuracy')
scores2 = cross_val_score(clasificador2, df1, target1, cv=10, scoring='accuracy')
scores3 = cross_val_score(clasificador3, df1, target1, cv=10, scoring='accuracy')

print("[Clasificador1] Mean: %0.3f - Standar Deviation: %0.3f"%(scores1.mean(), scores1.std()))
print("[Clasificador2] Mean: %0.3f - Standar Deviation: %0.3f"%(scores2.mean(), scores2.std()))
print("[Clasificador3] Mean: %0.3f - Standar Deviation: %0.3f"%(scores3.mean(), scores3.std()))

[Clasificador1] Mean: 0.761 - Standar Deviation: 0.181
[Clasificador2] Mean: 0.840 - Standar Deviation: 0.042
[Clasificador3] Mean: 0.837 - Standar Deviation: 0.079


# Pima Indian Diabetes

1. Number of times pregnant
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
3. Diastolic blood pressure (mm Hg)
4. Triceps skin fold thickness (mm)
5. 2-Hour serum insulin (mu U/ml)
6. Body mass index (weight in kg/(height in m)^2)
7. Diabetes pedigree function
8. Age (years)
9. Class variable (0 or 1)

In [18]:
df2 = pd.read_csv('data/diabetes.data')

In [19]:
#Quitar las filas que tengan missing values '?'
index = []
for i in range(df2.shape[0]):
    if('?' in df2.iloc[i].values):
        index.append(i)
df2 = df2.drop(index)  

In [20]:
target2 = df2['Class']
df2 = df2.drop('Class',axis=1)
mat = df2.as_matrix()

## Clustering

### K-Means

In [21]:
kmeans = cluster.KMeans(n_clusters=2)
kmeans.fit(mat)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [22]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, kmeans.labels_))

Silhouette Coefficient: 0.568790


### Average Linkage

In [23]:
algoritmo = cluster.AgglomerativeClustering(linkage="average", 
                                            affinity="cityblock", n_clusters=2)
# Linkage: complete, average, ward
# Affinity: “euclidean”, “l1”, “l2”, “manhattan”, "cityblock", “cosine”, o ‘precomputed’
algoritmo.fit(mat)

AgglomerativeClustering(affinity='cityblock', compute_full_tree='auto',
            connectivity=None, linkage='average', memory=None,
            n_clusters=2, pooling_func=<function mean at 0x7f5454415400>)

In [24]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, algoritmo.labels_))

Silhouette Coefficient: 0.728518


### Mean-Shift

In [25]:
ancho_banda = cluster.estimate_bandwidth(mat, quantile=0.15)
mean_shift_alg = cluster.MeanShift(bandwidth=ancho_banda, bin_seeding=True)
mean_shift_alg.fit(mat)

MeanShift(bandwidth=63.883927652192064, bin_seeding=True, cluster_all=True,
     min_bin_freq=1, n_jobs=1, seeds=None)

In [26]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, mean_shift_alg.labels_))

Silhouette Coefficient: 0.428159


## Clasificación

In [27]:
D_entrenamiento, D_test, target2_entrenamiento, target2_test = train_test_split(df2, target2, test_size = 0.5, random_state=0)

sc2 = StandardScaler()
sc2.fit(D_entrenamiento) 
Z2_entrenamiento = sc2.transform(D_entrenamiento)
Z2_test = sc2.transform(D_test)

clasificador1 = LinearDiscriminantAnalysis()
clasificador1.fit(Z2_entrenamiento, target2_entrenamiento)
target2_pred1 = clasificador1.predict(Z2_test)
accuracy1 = accuracy_score(target2_test, target2_pred1)
precision1 = precision_score(target2_test, target2_pred1, average="macro")
F1 = f1_score(target2_test, target2_pred1, average='macro')

clasificador2 = QuadraticDiscriminantAnalysis()
clasificador2.fit(Z2_entrenamiento, target2_entrenamiento)
target2_pred2 = clasificador2.predict(Z2_test)
accuracy2 = accuracy_score(target2_test, target2_pred2)
precision2 = precision_score(target2_test, target2_pred2, average="macro")
F2 = f1_score(target2_test, target2_pred2, average='macro')

clasificador3=SVC(kernel="linear", C=0.025)
clasificador3.fit(Z2_entrenamiento, target2_entrenamiento)
target2_pred3 = clasificador3.predict(Z2_test)
accuracy3 = accuracy_score(target2_test, target2_pred3)
precision3 = precision_score(target2_test, target2_pred3, average="macro")
F3 = f1_score(target2_test, target2_pred3, average='macro')

print("[Clasificador 1] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy1, precision1, F1))
print("[Clasificador 2] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy2, precision2, F2))
print("[Clasificador 3] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy3, precision3, F3))

[Clasificador 1] Accuracy: 0.781  -  Precision: 0.77  -  F Score: 0.741
[Clasificador 2] Accuracy: 0.766  -  Precision: 0.74  -  F Score: 0.728
[Clasificador 3] Accuracy: 0.789  -  Precision: 0.79  -  F Score: 0.743


In [28]:
#KFold
scores1 = cross_val_score(clasificador1, df2, target2, cv=10, scoring='accuracy')
scores2 = cross_val_score(clasificador2, df2, target2, cv=10, scoring='accuracy')
scores3 = cross_val_score(clasificador3, df2, target2, cv=10, scoring='accuracy')

print("[Clasificador1] Mean: %0.3f - Standar Deviation: %0.3f"%(scores1.mean(), scores1.std()))
print("[Clasificador2] Mean: %0.3f - Standar Deviation: %0.3f"%(scores2.mean(), scores2.std()))
print("[Clasificador3] Mean: %0.3f - Standar Deviation: %0.3f"%(scores3.mean(), scores3.std()))

[Clasificador1] Mean: 0.773 - Standar Deviation: 0.035
[Clasificador2] Mean: 0.740 - Standar Deviation: 0.028
[Clasificador3] Mean: 0.766 - Standar Deviation: 0.025


## PCA

In [29]:
df2 = df2.astype(float)
df2 = (df2 - df2.mean()) / df2.std()
pca2 = pcasvd(df2, keepdim=6, demean=False)

In [30]:
D_entrenamiento, D_test, target2_entrenamiento, target2_test = train_test_split(pca2[0], target2, test_size = 0.5, random_state=0)

sc2 = StandardScaler()
sc2.fit(D_entrenamiento) 
Z2_entrenamiento = sc2.transform(D_entrenamiento)
Z2_test = sc2.transform(D_test)

clasificador1 = LinearDiscriminantAnalysis()
clasificador1.fit(Z2_entrenamiento, target2_entrenamiento)
target2_pred1 = clasificador1.predict(Z2_test)
accuracy1 = accuracy_score(target2_test, target2_pred1)
precision1 = precision_score(target2_test, target2_pred1, average="macro")
F1 = f1_score(target2_test, target2_pred1, average='macro')

clasificador2 = QuadraticDiscriminantAnalysis()
clasificador2.fit(Z2_entrenamiento, target2_entrenamiento)
target2_pred2 = clasificador2.predict(Z2_test)
accuracy2 = accuracy_score(target2_test, target2_pred2)
precision2 = precision_score(target2_test, target2_pred2, average="macro")
F2 = f1_score(target2_test, target2_pred2, average='macro')

clasificador3=SVC(kernel="linear", C=0.025)
clasificador3.fit(Z2_entrenamiento, target2_entrenamiento)
target2_pred3 = clasificador3.predict(Z2_test)
accuracy3 = accuracy_score(target2_test, target2_pred3)
precision3 = precision_score(target2_test, target2_pred3, average="macro")
F3 = f1_score(target2_test, target2_pred3, average='macro')

print("[Clasificador 1] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy1, precision1, F1))
print("[Clasificador 2] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy2, precision2, F2))
print("[Clasificador 3] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy3, precision3, F3))

[Clasificador 1] Accuracy: 0.781  -  Precision: 0.77  -  F Score: 0.741
[Clasificador 2] Accuracy: 0.688  -  Precision: 0.64  -  F Score: 0.626
[Clasificador 3] Accuracy: 0.781  -  Precision: 0.77  -  F Score: 0.738


In [31]:
#KFold
scores1 = cross_val_score(clasificador1, df2, target2, cv=10, scoring='accuracy')
scores2 = cross_val_score(clasificador2, df2, target2, cv=10, scoring='accuracy')
scores3 = cross_val_score(clasificador3, df2, target2, cv=10, scoring='accuracy')

print("[Clasificador1] Mean: %0.3f - Standar Deviation: %0.3f"%(scores1.mean(), scores1.std()))
print("[Clasificador2] Mean: %0.3f - Standar Deviation: %0.3f"%(scores2.mean(), scores2.std()))
print("[Clasificador3] Mean: %0.3f - Standar Deviation: %0.3f"%(scores3.mean(), scores3.std()))

[Clasificador1] Mean: 0.773 - Standar Deviation: 0.035
[Clasificador2] Mean: 0.740 - Standar Deviation: 0.028
[Clasificador3] Mean: 0.771 - Standar Deviation: 0.033


# Water Treatment Plant

|N. | Identificador | Descripción |
| -- |:---:|:---|
| 1 | Q-E       | (input flow to plant)   |
| 2 | ZN-E      | (input Zinc to plant) |
| 3 | PH-E      | (input pH to plant)  |
| 4 | DBO-E     | (input Biological demand of oxygen to plant)  |
| 5 | DQO-E     | (input chemical demand of oxygen to plant) |
| 6 | SS-E      | (input suspended solids to plant)   |
| 7 | SSV-E     | (input volatile supended solids to plant) |
| 8 | SED-E     | (input sediments to plant)  |
| 9 | COND-E    | (input conductivity to plant)  |
|10 | PH-P      | (input pH to primary settler) |
|11 | DBO-P     | (input Biological demand of oxygen to primary settler) |
|12 | SS-P      | (input suspended solids to primary settler) |
|13 | SSV-P     | (input volatile supended solids to primary settler) |
|14 | SED-P     | (input sediments to primary settler)  |
|15 | COND-P    | (input conductivity to primary settler) |
|16 | PH-D      | (input pH to secondary settler)  |
|17 | DBO-D     | (input Biological demand of oxygen to secondary settler) |
|18 | DQO-D     | (input chemical demand of oxygen to secondary settler) |
|19 | SS-D      | (input suspended solids to secondary settler) |
|20 | SSV-D     | (input volatile supended solids to secondary settler) |
|21 | SED-D     | (input sediments to secondary settler)   |
|22 | COND-D    | (input conductivity to secondary settler)  |
|23 | PH-S      | (output pH)    |
|24 | DBO-S     | (output Biological demand of oxygen) |
|25 | DQO-S     | (output chemical demand of oxygen) |
|26 | SS-S      | (output suspended solids) |
|27 | SSV-S     | (output volatile supended solids)  |
|28 | SED-S     | (output sediments)  |
|29 | COND-S    | (output conductivity) |
|30 | RD-DBO-P  | (performance input Biological demand of oxygen in primary settler) |
|31 | RD-SS-P   | (performance input suspended solids to primary settler) |
|32 | RD-SED-P  | (performance input sediments to primary settler) |
|33 | RD-DBO-S  | (performance input Biological demand of oxygen to secondary settler) |
|34 | RD-DQO-S  | (performance input chemical demand of oxygen to secondary settler) |
|35 | RD-DBO-G  | (global performance input Biological demand of oxygen) |
|36 | RD-DQO-G  | (global performance input chemical demand of oxygen) |
|37 | RD-SS-G   | (global performance input suspended solids)  |
|38 | RD-SED-G  | (global performance input sediments) |

Hay 13 clases a las que puede pertenecer esto

* Class 1: Normal situation
* Class 2: Secondary settler problems-1
* Class 3: Secondary settler problems-2
* Class 4: Secondary settler problems-3
* Class 5: Normal situation with performance over the mean
* Class 6: Solids overload-1
* Class 7: Secondary settler problems-4
* Class 8: Storm-1
* Class 9: Normal situation with a low influent
* Class 10: Storm-2
* Class 11: Normal situation
* Class 12: Storm-3
* Class 13: Solids overload-2

In [32]:
df3 = pd.read_csv('data/water-treatment.data')

In [33]:
#Quitar las filas que tengan missing values '?'
index = []
for j in df3.index:
    if('?' in df3.loc[j].values):
        index.append(j)
df3 = df3.drop(index)      

In [34]:
np.random.seed(1)
target3 = np.random.randint(2,13,size=380)
#dataset no incluye el atributo de la clase
mat = df3.as_matrix()

## Clustering

### K-Means

In [35]:
kmeans = cluster.KMeans(n_clusters=13)
kmeans.fit(mat)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=13, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [36]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, kmeans.labels_))

Silhouette Coefficient: 0.381638


### Average Linkage

In [37]:
algoritmo = cluster.AgglomerativeClustering(linkage="average", 
                                            affinity="cityblock", n_clusters=13)
# Linkage: complete, average, ward
# Affinity: “euclidean”, “l1”, “l2”, “manhattan”, "cityblock", “cosine”, o ‘precomputed’
algoritmo.fit(mat)

AgglomerativeClustering(affinity='cityblock', compute_full_tree='auto',
            connectivity=None, linkage='average', memory=None,
            n_clusters=13, pooling_func=<function mean at 0x7f5454415400>)

In [38]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, algoritmo.labels_))

Silhouette Coefficient: 0.342311


### Mean-Shift

In [39]:
ancho_banda = cluster.estimate_bandwidth(mat, quantile=0.15)
mean_shift_alg = cluster.MeanShift(bandwidth=ancho_banda, bin_seeding=True)
mean_shift_alg.fit(mat)

MeanShift(bandwidth=2703.5498315775735, bin_seeding=True, cluster_all=True,
     min_bin_freq=1, n_jobs=1, seeds=None)

In [40]:
print("Silhouette Coefficient: %0.6f"
      % metrics.silhouette_score(mat, mean_shift_alg.labels_))

Silhouette Coefficient: 0.512379


## Clasificación

In [41]:
W_entrenamiento, W_test, target3_entrenamiento, target3_test = train_test_split(df3, target3, test_size = 0.5, random_state=0)

sc3 = StandardScaler()
sc3.fit(W_entrenamiento) 
Z3_entrenamiento = sc3.transform(W_entrenamiento)
Z3_test = sc3.transform(W_test)

clasificador1 = LinearDiscriminantAnalysis()
clasificador1.fit(Z3_entrenamiento, target3_entrenamiento)
target3_pred1 = clasificador1.predict(Z3_test)
accuracy1 = accuracy_score(target3_test, target3_pred1)
precision1 = precision_score(target3_test, target3_pred1, average="macro")
F1 = f1_score(target3_test, target3_pred1, average='macro')

clasificador2 = QuadraticDiscriminantAnalysis()
clasificador2.fit(Z3_entrenamiento, target3_entrenamiento)
target3_pred2 = clasificador2.predict(Z3_test)
accuracy2 = accuracy_score(target3_test, target3_pred2)
precision2 = precision_score(target3_test, target3_pred2, average="macro")
F2 = f1_score(target3_test, target3_pred2, average='macro')

clasificador3=SVC(kernel="linear", C=0.025)
clasificador3.fit(Z3_entrenamiento, target3_entrenamiento)
target3_pred3 = clasificador3.predict(Z3_test)
accuracy3 = accuracy_score(target3_test, target3_pred3)
precision3 = precision_score(target3_test, target3_pred3, average="macro")
F3 = f1_score(target3_test, target3_pred3, average='macro')

print("[Clasificador 1] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy1, precision1, F1))
print("[Clasificador 2] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy2, precision2, F2))
print("[Clasificador 3] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy3, precision3, F3))

[Clasificador 1] Accuracy: 0.116  -  Precision: 0.10  -  F Score: 0.103
[Clasificador 2] Accuracy: 0.095  -  Precision: 0.11  -  F Score: 0.095
[Clasificador 3] Accuracy: 0.084  -  Precision: 0.11  -  F Score: 0.078


In [ ]:
#KFold
scores1 = cross_val_score(clasificador1, df3, target3, cv=10, scoring='accuracy')
scores2 = cross_val_score(clasificador2, df3, target3, cv=10, scoring='accuracy')
scores3 = cross_val_score(clasificador3, df3, target3, cv=10, scoring='accuracy')

print("[Clasificador1] Mean: %0.3f - Standar Deviation: %0.3f"%(scores1.mean(), scores1.std()))
print("[Clasificador2] Mean: %0.3f - Standar Deviation: %0.3f"%(scores2.mean(), scores2.std()))
print("[Clasificador3] Mean: %0.3f - Standar Deviation: %0.3f"%(scores3.mean(), scores3.std()))

## PCA

In [ ]:
df3 = df3.astype(float)
df3 = (df3 - df3.mean()) / df3.std()
pca3 = pcasvd(df3, keepdim=30, demean=False)

In [ ]:
W_entrenamiento, W_test, target3_entrenamiento, target3_test = train_test_split(pca3[0], target3, test_size = 0.5, random_state=0)

sc3 = StandardScaler()
sc3.fit(W_entrenamiento) 
Z3_entrenamiento = sc3.transform(W_entrenamiento)
Z3_test = sc3.transform(W_test)

clasificador1 = LinearDiscriminantAnalysis()
clasificador1.fit(Z3_entrenamiento, target3_entrenamiento)
target3_pred1 = clasificador1.predict(Z3_test)
accuracy1 = accuracy_score(target3_test, target3_pred1)
precision1 = precision_score(target3_test, target3_pred1, average="macro")
F1 = f1_score(target3_test, target3_pred1, average='macro')

clasificador2 = QuadraticDiscriminantAnalysis()
clasificador2.fit(Z3_entrenamiento, target3_entrenamiento)
target3_pred2 = clasificador2.predict(Z3_test)
accuracy2 = accuracy_score(target3_test, target3_pred2)
precision2 = precision_score(target3_test, target3_pred2, average="macro")
F2 = f1_score(target3_test, target3_pred2, average='macro')

clasificador3=SVC(kernel="linear", C=0.025)
clasificador3.fit(Z3_entrenamiento, target3_entrenamiento)
target3_pred3 = clasificador3.predict(Z3_test)
accuracy3 = accuracy_score(target3_test, target3_pred3)
precision3 = precision_score(target3_test, target3_pred3, average="macro")
F3 = f1_score(target3_test, target3_pred3, average='macro')

print("[Clasificador 1] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy1, precision1, F1))
print("[Clasificador 2] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy2, precision2, F2))
print("[Clasificador 3] Accuracy: %0.3f  -  Precision: %0.2f  -  F Score: %0.3f"%(accuracy3, precision3, F3))

In [ ]:
#KFold
scores1 = cross_val_score(clasificador1, df3, target3, cv=10, scoring='accuracy')
scores2 = cross_val_score(clasificador2, df3, target3, cv=10, scoring='accuracy')
scores3 = cross_val_score(clasificador3, df3, target3, cv=10, scoring='accuracy')

print("[Clasificador1] Mean: %0.3f - Standar Deviation: %0.3f"%(scores1.mean(), scores1.std()))
print("[Clasificador2] Mean: %0.3f - Standar Deviation: %0.3f"%(scores2.mean(), scores2.std()))
print("[Clasificador3] Mean: %0.3f - Standar Deviation: %0.3f"%(scores3.mean(), scores3.std()))